In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer, BloomForSequenceClassification, TrainingArguments, Trainer
from torch.utils.tensorboard import SummaryWriter
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig 
import loralib as lora
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pyarrow as pa
from datasets import Dataset

/home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def process_data(row, tokenizer):

    text = row['sentence']
    text = str(text)

    encodings = tokenizer(text, padding=True)

    encodings['label'] = row['label_for_kaggle']
    encodings['text'] = text

    return encodings

In [5]:
model_type = 'ckip-joint/bloom-1b1-zh'
tokenizer = AutoTokenizer.from_pretrained(model_type)
model = BloomForSequenceClassification.from_pretrained(model_type, num_labels=2)
device = torch.device('cuda:0')

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()
model = model.to(device)
model.train()

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at ckip-joint/bloom-1b1-zh and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1185792 || all params: 1066500096 || trainable%: 0.1111853627062402


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BloomForSequenceClassification(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1536)
        (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0): BloomBlock(
            (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                in_features=1536, out_features=4608, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4608, bias=False)
                )
              )
              

In [6]:
train_file = pd.read_csv('./COS_train.csv')
test_file = pd.read_csv('./COS_test.csv')
processed_data = []

for i in range(len(train_file)):
    processed_data.append(process_data(train_file.iloc[i], tokenizer))

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [7]:
training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

In [8]:
trainer.train()


/home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 4
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 4
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/author

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/wandb/sdk/wandb_init.py:1129 in   │
│ init                                                                                             │
│                                                                                                  │
│   1126 │   except_exit = None                                                                    │
│   1127 │   try:                                                                                  │
│   1128 │   │   wi = _WandbInit()                                                                 │
│ ❱ 1129 │   │   wi.setup(kwargs)                                                                  │
│   1130 │   │   assert wi.settings                                                                │
│   1131 │   │   except_exit = wi.settings._except_exit                                            │
│   1132 │   │   try:                                                                              │
│                                                                                                  │
│ /home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/wandb/sdk/wandb_init.py:274 in    │
│ setup                                                                                            │
│                                                                                                  │
│    271 │   │   │   settings.update(init_settings, source=Source.INIT)                            │
│    272 │   │                                                                                     │
│    273 │   │   if not settings._offline and not settings._noop:                                  │
│ ❱  274 │   │   │   wandb_login._login(                                                           │
│    275 │   │   │   │   anonymous=kwargs.pop("anonymous", None),                                  │
│    276 │   │   │   │   force=kwargs.pop("force", None),                                          │
│    277 │   │   │   │   _disable_warning=True,                                                    │
│                                                                                                  │
│ /home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/wandb/sdk/wandb_login.py:298 in   │
│ _login                                                                                           │
│                                                                                                  │
│   295 │   │   return logged_in                                                                   │
│   296 │                                                                                          │
│   297 │   if not key:                                                                            │
│ ❱ 298 │   │   wlogin.prompt_api_key()                                                            │
│   299 │                                                                                          │
│   300 │   # make sure login credentials get to the backend                                       │
│   301 │   wlogin.propogate_login()                                                               │
│                                                                                                  │
│ /home/yihan/miniconda3/envs/hongfa/lib/python3.9/site-packages/wandb/sdk/wandb_login.py:221 in   │
│ prompt_api_key                                                                                   │
│                                                                                                  │
│   218 │   │   │   return key, ApiKeyStatus.VALID                                                 │
│   219 │                                                                                          │
│   220 │   def prompt_api_key(self):                        

: 

In [ ]:
trainer.evaluate()